Importing necessary libraries for machine learning

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Importing the dataset

In [2]:
df = pd.read_excel("Final_CI_CD_data.xlsx", sheet_name="CI")

Dropping unnecessary columns

In [3]:
df = df.drop(['Source','Rock Name', 'Magnesite (%)', 'Dolomite (%)', 'Gypsum (%)', 'Anhydrite (%)', 'UCS (MPa)'], axis=1)

Shuffling the dataset for randomness

In [4]:
df = df.sample(frac=1, random_state=13).reset_index(drop=True)
df

,Mean Grain Size (mm),Rock Classification,Plagioclase feldspar (%),Alkali feldspar (%),Quartz (%),Calcite (%),Clay (%),Mica (%),Amphibole (%),Density (g/cm3),Porosity (%),E (GPa),v,CI (MPa)
0,3.500,Igneous,38.0,27.0,31.0,0,0.0,4.0,0.0,2.620,0.260,66.100,0.31,71.20
1,3.500,Igneous,38.0,27.0,31.0,0,0.0,4.0,0.0,2.620,0.260,66.100,0.31,66.00
2,3.500,Igneous,38.0,27.0,31.0,0,0.0,4.0,0.0,2.620,0.260,66.100,0.31,59.10
3,3.000,Igneous,34.0,23.0,36.0,0,0.0,6.0,0.0,2.650,0.400,72.600,0.26,126.50
4,0.002,Sedimentary,0.0,4.0,15.0,15,60.0,0.0,0.0,2.150,18.000,2.134,0.16,2.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,2.000,Metamorphic,17.0,11.0,34.0,0,0.0,26.0,0.0,2.724,0.183,68.070,0.23,51.00
1340,2.500,Igneous,34.0,25.0,22.0,0,0.0,12.0,7.0,2.640,0.960,51.700,0.19,58.68
1341,1.500,Igneous,73.0,8.0,19.0,0,0.0,0.0,0.0,2.740,0.400,73.000,0.29,157.00
1342,1.750,Igneous,44.0,20.0,22.0,0,0.0,9.0,0.0,2.685,0.240,73.000,0.12,86.00


## DATA TRANSFORMATION

Applying one-hot encoding to convert qualitative feature "Rock Classification" into three numerical binary columns

In [5]:

def one_hot_encode(df, column_name):
    """
    Perform one-hot encoding on a specified column of a Pandas DataFrame.
    
    Parameters:
    -----------
    df : pandas DataFrame
        The DataFrame to be encoded.
    column_name : str
        The name of the column to be encoded.
    
    Returns:
    --------
    pandas DataFrame
        The encoded DataFrame.
    """
    # Create a new DataFrame with the one-hot encoded columns
    encoded_cols = pd.get_dummies(df[column_name], prefix=column_name)
    
    # Concatenate the original DataFrame with the encoded columns
    df_encoded = pd.concat([df, encoded_cols], axis=1)
    
    # Drop the original categorical column
    df_encoded.drop(column_name, axis=1, inplace=True)
    
    return df_encoded

In [6]:
df = one_hot_encode(df,"Rock Classification")

Changing the column name of new columns to "IGN", "MET", "SED" for igneous, metamorphic and sedimentary, respectively.

In [7]:
num_columns = len(df.columns)
last_three_columns = df.iloc[:, num_columns - 3:]
first_columns = df.iloc[:, :num_columns - 3]

# Concatenate the last three columns with the first columns
df = pd.concat([last_three_columns, first_columns], axis=1)
new_column_names = ['IGN', 'MET', 'SED']
df = df.rename(columns=dict(zip(df.columns[:3], new_column_names)))
df

,IGN,MET,SED,Mean Grain Size (mm),Plagioclase feldspar (%),Alkali feldspar (%),Quartz (%),Calcite (%),Clay (%),Mica (%),Amphibole (%),Density (g/cm3),Porosity (%),E (GPa),v,CI (MPa)
0,1,0,0,3.500,38.0,27.0,31.0,0,0.0,4.0,0.0,2.620,0.260,66.100,0.31,71.20
1,1,0,0,3.500,38.0,27.0,31.0,0,0.0,4.0,0.0,2.620,0.260,66.100,0.31,66.00
2,1,0,0,3.500,38.0,27.0,31.0,0,0.0,4.0,0.0,2.620,0.260,66.100,0.31,59.10
3,1,0,0,3.000,34.0,23.0,36.0,0,0.0,6.0,0.0,2.650,0.400,72.600,0.26,126.50
4,0,0,1,0.002,0.0,4.0,15.0,15,60.0,0.0,0.0,2.150,18.000,2.134,0.16,2.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,0,1,0,2.000,17.0,11.0,34.0,0,0.0,26.0,0.0,2.724,0.183,68.070,0.23,51.00
1340,1,0,0,2.500,34.0,25.0,22.0,0,0.0,12.0,7.0,2.640,0.960,51.700,0.19,58.68
1341,1,0,0,1.500,73.0,8.0,19.0,0,0.0,0.0,0.0,2.740,0.400,73.000,0.29,157.00
1342,1,0,0,1.750,44.0,20.0,22.0,0,0.0,9.0,0.0,2.685,0.240,73.000,0.12,86.00


Normalising all columns so that each one's value fall between 0 and 1.

In [8]:
def normalize_numeric_columns(df, dependent_col):
    """
    Normalize all numerical columns in a Pandas DataFrame, except for a specified dependent column.
    
    Parameters:
    -----------
    df : pandas DataFrame
        The DataFrame to be normalized.
    dependent_col : str
        The name of the dependent column (i.e., the column to be predicted) that should not be normalized.
    
    Returns:
    --------
    pandas DataFrame
        The normalized DataFrame.
    """
    # Get the names of all numerical columns except for the dependent column
    numeric_cols = [col for col in df.columns if col != dependent_col and pd.api.types.is_numeric_dtype(df[col])]
    
    # Create a scaler object to normalize the data
    scaler = MinMaxScaler()
    
    # Normalize the numerical columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    
    return df

In [9]:
df = normalize_numeric_columns(df, "CI_MPa")

In [10]:
df

,IGN,MET,SED,Mean Grain Size (mm),Plagioclase feldspar (%),Alkali feldspar (%),Quartz (%),Calcite (%),Clay (%),Mica (%),Amphibole (%),Density (g/cm3),Porosity (%),E (GPa),v,CI (MPa)
0,1.0,0.0,0.0,0.174917,0.520548,0.465517,0.326316,0.00,0.00000,0.100,0.0000,0.670330,0.007667,0.723293,0.642857,0.369673
1,1.0,0.0,0.0,0.174917,0.520548,0.465517,0.326316,0.00,0.00000,0.100,0.0000,0.670330,0.007667,0.723293,0.642857,0.342313
2,1.0,0.0,0.0,0.174917,0.520548,0.465517,0.326316,0.00,0.00000,0.100,0.0000,0.670330,0.007667,0.723293,0.642857,0.306009
3,1.0,0.0,0.0,0.149915,0.465753,0.396552,0.378947,0.00,0.00000,0.150,0.0000,0.703297,0.013171,0.795526,0.523810,0.660633
4,0.0,0.0,1.0,0.000000,0.000000,0.068966,0.157895,0.15,0.84507,0.000,0.0000,0.153846,0.705131,0.012457,0.285714,0.006314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,0.0,1.0,0.0,0.099910,0.232877,0.189655,0.357895,0.00,0.00000,0.650,0.0000,0.784615,0.004639,0.745185,0.452381,0.263391
1340,1.0,0.0,0.0,0.124912,0.465753,0.431034,0.231579,0.00,0.00000,0.300,0.0875,0.692308,0.035188,0.563270,0.357143,0.303799
1341,1.0,0.0,0.0,0.074907,1.000000,0.137931,0.200000,0.00,0.00000,0.000,0.0000,0.802198,0.013171,0.799971,0.595238,0.821109
1342,1.0,0.0,0.0,0.087409,0.602740,0.344828,0.231579,0.00,0.00000,0.225,0.0000,0.741758,0.006880,0.799971,0.190476,0.447543


## HYPERPARAMETRIC TUNING AND MODEL SELECTION

Importing machine learning algorithms

In [11]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
lr = LinearRegression()
ridge = Ridge()
lasso = Lasso()
en = ElasticNet()
svr = SVR()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()
abr = AdaBoostRegressor()
gbr = GradientBoostingRegressor()
xgbr = XGBRegressor()

Splitting the dataset into dependent and independent variables

In [12]:
X_train, y_train  = df.drop(["CI (MPa)"], axis=1), df["CI (MPa)"]

Tuning Linear Regression

In [13]:
grid_lr = GridSearchCV(estimator=lr, param_grid={"fit_intercept" : [True, False],
                                                 }, 
                                                 scoring="r2",
                                                 cv=4)

In [14]:
grid_lr.fit(X_train, y_train)
grid_lr.best_params_, grid_lr.best_score_

({'fit_intercept': False}, 0.6705520398579727)

In [15]:
f"R2 Score : {np.round(grid_lr.best_score_*100,3)}"

'R2 Score : 67.055'

Tuning Ridge Regression

In [16]:
grid_ridge = GridSearchCV(estimator=ridge, param_grid={"fit_intercept" : [True, False],
                                                        'alpha': np.logspace(-4, 4, 9),
                                                        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']},
                                                        scoring="r2",
                                                        cv=4)

In [17]:
grid_ridge.fit(X_train, y_train)
grid_ridge.best_params_, grid_ridge.best_score_

({'alpha': 1.0, 'fit_intercept': False, 'solver': 'svd'}, 0.6714643993090245)

In [18]:
f"R2 Score : {np.round(grid_ridge.best_score_*100,3)}"

'R2 Score : 67.146'

Tuning Lasso Regression

In [19]:
grid_lasso = GridSearchCV(estimator=lasso, param_grid={"fit_intercept" : [True, False],
                                                        'alpha': np.logspace(-3, 3, 7),
                                                        'tol': [1e-5, 1e-4, 1e-3]}, 
                                                        scoring="r2",
                                                        cv=4)

In [20]:
grid_lasso.fit(X_train, y_train)
grid_lasso.best_params_, grid_lasso.best_score_

({'alpha': 0.001, 'fit_intercept': False, 'tol': 0.001}, 0.6579542954822035)

In [21]:
f"R2 Score : {np.round(grid_lasso.best_score_*100,3)}"

'R2 Score : 65.795'

Tuning ElasticNet Regression

In [22]:
grid_en = GridSearchCV(estimator=en, param_grid={"fit_intercept" : [True, False],
                                                        'alpha': np.logspace(-4, 0, 50), # range of values for l1_ratio
                                                        'tol': [1e-3, 1e-4, 1e-5]}, # range of values for tol, 
                                                        scoring="r2",
                                                        cv=4)

In [23]:
grid_en.fit(X_train, y_train)
grid_en.best_params_, grid_en.best_score_

({'alpha': 0.00014563484775012445, 'fit_intercept': False, 'tol': 1e-05},
 0.6720001269003869)

In [24]:
f"R2 Score : {np.round(grid_en.best_score_*100,3)}"

'R2 Score : 67.2'

Tuning Support Vector Regressor

In [25]:
grid_svr = GridSearchCV(estimator=svr, param_grid={"kernel":['linear', 'poly', 'rbf', 'sigmoid'],
                                                    }, 
                                                        scoring="r2",
                                                        cv=4)

In [26]:
grid_svr.fit(X_train, y_train)
grid_svr.best_params_, grid_svr.best_score_

({'kernel': 'rbf'}, 0.7344156498855053)

In [27]:
f"R2 Score : {np.round(grid_svr.best_score_*100,3)}"

'R2 Score : 73.442'

Tuning Decision Tree regressor

In [28]:
grid_dtr = GridSearchCV(estimator=dtr, 
                        param_grid={"criterion":['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                                    "splitter": ["best", "random"],
                                    "max_depth": [3, 5, 7, 10, 15, 25]}, 
                        
                        scoring="r2",
                        cv=4)

In [29]:
grid_dtr.fit(X_train, y_train)
grid_dtr.best_params_, grid_dtr.best_score_

({'criterion': 'friedman_mse', 'max_depth': 5, 'splitter': 'best'},
 0.7560206527200402)

In [30]:
f"R2 Score : {np.round(grid_dtr.best_score_*100,3)}"

'R2 Score : 75.602'

Tuning Random Forest Regressor

In [31]:
grid_rfr = GridSearchCV(estimator=rfr, 
                        param_grid={"criterion":['squared_error', 'absolute_error', 'poisson'],
                                    "max_depth": [3, 5, 7, 10, 15, 25],
                                     },
                        scoring="r2",
                        cv=4)

In [32]:
grid_rfr.fit(X_train, y_train)
grid_rfr.best_params_, grid_rfr.best_score_

({'criterion': 'poisson', 'max_depth': 25}, 0.5798349212753207)

In [33]:
f"R2 Score : {np.round(grid_rfr.best_score_*100,3)}"

'R2 Score : 57.983'

Tuning Adaboost regressor

In [34]:
grid_abr = GridSearchCV(estimator=abr, 
                        param_grid={"loss":['linear', 'square', 'exponential'],
                                     },  
                        scoring="r2",
                        cv=4)

In [35]:
grid_abr.fit(X_train, y_train)
grid_abr.best_params_, grid_abr.best_score_

({'loss': 'exponential'}, 0.6961927307028789)

In [36]:
f"R2 Score : {np.round(grid_abr.best_score_*100,3)}"

'R2 Score : 69.619'

Tuning Gradient Boosting Regressor

In [37]:
grid_gbr = GridSearchCV(estimator=gbr, 
                        param_grid={"loss":['squared_error', 'absolute_error', 'huber', 'quantile'],
                                    "criterion":['friedman_mse', 'squared_error', 'mse']}, 
                        scoring="r2",
                        cv=4)

In [38]:
grid_gbr.fit(X_train, y_train)
grid_gbr.best_params_, grid_gbr.best_score_

({'criterion': 'mse', 'loss': 'huber'}, 0.7986693680173448)

In [39]:
f"R2 Score : {np.round(grid_gbr.best_score_*100,3)}"

'R2 Score : 79.867'

Tuning XGBoost regressor

In [40]:
grid_xgbr = GridSearchCV(estimator=xgbr, 
                        param_grid={
                                    "max_depth": [3, 5, 7, 10, 15, 25],
                                     'learning_rate': [0.1, 0.01]}, 
                        scoring="r2",
                        cv=4)

In [41]:
grid_xgbr.fit(X_train, y_train)
grid_xgbr.best_params_, grid_xgbr.best_score_

({'learning_rate': 0.1, 'max_depth': 3}, 0.8012149320771182)

In [42]:
f"R2 Score : {np.round(grid_xgbr.best_score_*100,3)}"

'R2 Score : 80.121'

| ML algorithm | R2_score |
| -------- | -------- |
| Linear Regression |67.05 |
| Ridge Regression | 67.14 |
| Lasso Regression | 65.79 |
| ElaasticNet Regression | 67.20 |
| Support Vector Regression | 73.44 |
| Decision Tree Regression | 76.43 |
| Random Forest Regression | 57.06 |
| AdaBoost Regression | 70.45 |
| GradientBoost Regression| 79.84 |
| XgBoost Regression | 80.12 |



## MODEL TRAINING

In [43]:
ci_model = XGBRegressor(learning_rate=0.1, max_depth=3)
ci_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)